#weightedcorrs.py - Python function to calculate weighted correlation coefficients, covariance, and standard deviations

SYNTAX

- results = weightedcorrs(X)
- results = weightedcorrs(X,w)

weightedcorrs returns a **results** dictionary that contains four outputs: **R**, **p**, **wcov**, and **wstd**.

**results['R']** is the output of the weighted Pearson correlation coefficients calculated from an input nobs-by-nvar matrix X whose rows are observations and whose columns are variables and an input nobs-by-1 vector w of weights for the observations. This function may be a valid alternative to np.corrcoef if observations are not all equally relevant and need to be weighted according to some theoretical hypothesis or knowledge.

**results['p']** is the output of the p-values of the Pearson correlation coefficients

**results['wcov']** is the output of the weighted covariance matrix

**results['wstd']** is the output of the weighted standard deviations

Input of w is optional.
If w=0, 1, or is omitted, then the function assigns w = np.ones(nobs).
If w=0 or omitted, then the covariance and standard deviations are unweighted and normalizd to nobs-1.
If w=1, then the covariance and standard deviations are unweighted and normlizd to nobs.
Otherwise, if w is an input vector of weights, then results are normalized to nobs for output of standard deviations and covariance.
If w=0 or omitted, or if the input vector of w = np.ones(nobs), then no difference exists between
weightedcorrs(X, w) and np.corrcoef(X,rowvar=False).

___

REFERENCE: the mathematical formulas in matrix notation, together with the original matlab code (weightedcorrs.m), is also available in F. Pozzi, T. Di Matteo, T. Aste, "Exponential smoothing weighted correlations", The European Physical Journal B, Volume 85, Issue 6, 2012. DOI:10.1140/epjb/e2012-20697-x. (https://www.researchgate.net/publication/257866466_Exponential_smoothing_weighted_correlations)
___

Adapted from weightedcorrs.m (Pozzi et al. 2012). Modified by Greg Pelletier 22-Jan-2024 to also output weighted covariance matrix and calculate weighted standard deviations, and allow optional input of weighting factors for use with unweighted analysis or normalization to nobs-1

___

First we will define the weightedcorrs function as follows:

In [1]:
def weightedcorrs(X,w=0):

  import sys
  import numpy as np
  import numpy.matlib
  import scipy.stats

  nobs,nvar = np.shape(X)   # nobs: number of observations; nvar: number of variables

  if np.size(w) == 1:
    if w==0:
      df_eq_nobs = False
    elif w==1:
      df_eq_nobs = True
    else:
      print('Error in weightcorrs: w needs be either unspecified, 0, 1, or a vector of real non-negative numbers with no infinite or nan values!','\n')
      sys.exit()
    w = np.ones(nobs)
  else:
    df_eq_nobs = True

  if not(np.shape(X)[0] == np.size(w)):
    print('Error in weightcorrs: np.shape(X)[0] has to be equal to np.size(w) (number of observations)!','\n')
    sys.exit()

  ctrl = w.ndim==1 and np.isreal(w).all() and (not np.isnan(w).any()) and (not np.isinf(w).any()) and np.all((w>=0)) and (np.sum(w)>0)
  if not ctrl:
    print('Error in weightcorrs: w needs be a vector of real non-negative numbers with no infinite or nan values!','\n')
    sys.exit()

  ctrl = np.isreal(X).all() and (not np.isnan(w).any()) and (not np.isinf(w).any()) and X.ndim==2
  if not ctrl:
    print('Error in weightcorrs: X needs be a 2D array of real non-negative numbers with no infinite or nan values!','\n')
    sys.exit()

  w = w / np.sum(w)
  temp = X - np.matlib.repmat(w @ X,nobs,1)              # remove mean (which is, also, weighted)
  temp = temp.T @ (temp * (np.matlib.repmat(w, nvar, 1)).T)    # weighted covariance
  wstd = np.sqrt(np.diag(temp))
  if df_eq_nobs == False:
    wstd = wstd * np.sqrt(nobs/(nobs-1));							# normalize to nobs-1 if w=0 or missing
  temp = 0.5 * (temp + temp.T)                        # must be exactly symmetric
  R = np.diag(temp)                                   # weighted variances normalized to nobs
  R = R.reshape(-1, 1)                                # reshape to vector column
  R = temp / np.sqrt(R @ R.T)                         # weighted correlation matrix
  D = np.diag(wstd)
  wcov = D @ R @ D															  	  # weighted covariance matrix

  dist = scipy.stats.beta(nobs/2 - 1, nobs/2 - 1, loc=-1, scale=2)
  p = 2*dist.cdf(-abs(R))                             # p-values of the correlation coefficients

  # put the outputs into the results dictionary
  results = {
    'R': R,
    'p': p,
    'wcov': wcov,
    'wstd': wstd
    }

  return results

Next we will show an example of using the weightedcorrs function. We will need to import numpy first

In [2]:
import numpy as np

We will use a data set from the MATLAB example data packages called 'hospital'. The X array will be a three column array, where the first column is the patient's weight, the second column is the systolic blood pressure, and the third column is the diastolic blood pressure.

Note that weightedcorrs requires the first index of X to be rows of observations, and the second index to be columns of random variables.

In [3]:
X = [np.array([176., 163., 131., 133., 119., 142., 142., 180., 183., 132., 128.,
        137., 174., 202., 129., 181., 191., 131., 179., 172., 133., 117.,
        137., 146., 123., 189., 143., 114., 166., 186., 126., 137., 138.,
        187., 193., 137., 192., 118., 180., 128., 164., 183., 169., 194.,
        172., 135., 182., 121., 158., 179., 170., 136., 135., 147., 186.,
        124., 134., 170., 180., 130., 130., 127., 141., 111., 134., 189.,
        137., 136., 130., 137., 186., 127., 176., 127., 115., 178., 131.,
        183., 194., 126., 186., 188., 189., 120., 132., 182., 120., 123.,
        141., 129., 184., 181., 124., 174., 134., 171., 188., 186., 172.,
        177.]),
 np.array([124., 109., 125., 117., 122., 121., 130., 115., 115., 118., 114.,
        115., 127., 130., 114., 130., 124., 123., 119., 125., 121., 123.,
        114., 128., 129., 114., 113., 125., 120., 127., 134., 121., 115.,
        127., 121., 127., 136., 117., 124., 120., 128., 116., 132., 137.,
        117., 116., 119., 123., 116., 124., 129., 130., 132., 117., 129.,
        118., 120., 138., 117., 113., 122., 115., 120., 117., 123., 123.,
        119., 110., 121., 138., 125., 122., 120., 117., 125., 124., 121.,
        118., 120., 118., 118., 122., 134., 131., 113., 125., 135., 128.,
        123., 122., 138., 124., 130., 123., 129., 128., 124., 119., 136.,
        114.]),
 np.array([93., 77., 83., 75., 80., 70., 88., 82., 78., 86., 77., 68., 74.,
        95., 79., 92., 95., 79., 77., 76., 75., 79., 88., 90., 96., 77.,
        80., 76., 83., 89., 92., 83., 80., 84., 92., 83., 90., 85., 90.,
        74., 92., 80., 89., 96., 89., 77., 81., 76., 83., 78., 95., 91.,
        91., 86., 89., 79., 74., 82., 76., 81., 77., 73., 85., 76., 80.,
        80., 79., 82., 79., 82., 75., 91., 74., 78., 85., 84., 75., 78.,
        81., 79., 85., 79., 82., 80., 80., 92., 92., 96., 87., 81., 90.,
        77., 91., 79., 73., 99., 92., 74., 93., 86.])]

# transpose X so that the first index is rows of observations (nobs), and the second index is columns of random variables (nvar)
X = np.transpose(X)

Next we will define the weighting factors to use for the analysis. In this example we will use the patients Age from the hospital data set as the weighting factors w.

In [4]:
w = np.array([38., 43., 38., 40., 49., 46., 33., 40., 28., 31., 45., 42., 25.,
       39., 36., 48., 32., 27., 37., 50., 48., 39., 41., 44., 28., 25.,
       39., 25., 36., 30., 45., 40., 25., 47., 44., 48., 44., 35., 33.,
       38., 39., 44., 44., 37., 45., 37., 30., 39., 42., 42., 49., 44.,
       43., 47., 50., 38., 41., 45., 36., 38., 29., 28., 30., 28., 29.,
       36., 45., 32., 31., 48., 25., 40., 39., 41., 33., 31., 35., 32.,
       42., 48., 34., 39., 28., 29., 32., 39., 37., 49., 31., 37., 38.,
       45., 30., 48., 48., 25., 44., 49., 45., 48.])

Now we are ready to show the example of using weightedcorrs to calculate the weighted correlation coefficients, weighted covariance, and weighted standard deviations as follows:

In [5]:
output = weightedcorrs(X,w)

R = output['R']
p = output['p']
wcov = output['wcov']
wstd = output['wstd']

print('weighted correlation coefficient matrix of X: ','\n',R,'\n')
print('p-values of the correlation coefficients: ','\n',p,'\n')
print('weighted covariance matrix of X: ','\n',wcov,'\n')
print('weighted standard deviations X: ','\n',wstd)

weighted correlation coefficient matrix of X:  
 [[1.         0.1554138  0.23071152]
 [0.1554138  1.         0.51036961]
 [0.23071152 0.51036961 1.        ]] 

p-values of the correlation coefficients:  
 [[0.00000000e+00 1.22589252e-01 2.09237757e-02]
 [1.22589252e-01 0.00000000e+00 5.81286900e-08]
 [2.09237757e-02 5.81286900e-08 0.00000000e+00]] 

weighted covariance matrix of X:  
 [[683.62291955  27.48984917  41.78750454]
 [ 27.48984917  45.76662876  23.91817879]
 [ 41.78750454  23.91817879  47.9885557 ]] 

weighted standard deviations X:  
 [26.14618365  6.76510375  6.92737726]


We can also use weightedcorrs to find the unweighted correlation coeeficients, covariance, and standard devations, normalized to nobs-1, by omitting the w argument (or by using 0 as the w argument) as follows:

In [6]:
output = weightedcorrs(X)

R = output['R']
p = output['p']
wcov = output['wcov']
wstd = output['wstd']

print('unweighted correlation coefficient matrix of X normalized to nobs-1: ','\n',R,'\n')
print('p-values of the correlation coefficients: ','\n',p,'\n')
print('unweighted covariance matrix of X normalized to nobs-1: ','\n',wcov,'\n')
print('unweighted standard deviations X normalized to nobs-1: ','\n',wstd)

unweighted correlation coefficient matrix of X normalized to nobs-1:  
 [[1.         0.15578811 0.22268743]
 [0.15578811 1.         0.51184337]
 [0.22268743 0.51184337 1.        ]] 

p-values of the correlation coefficients:  
 [[0.00000000e+00 1.21679870e-01 2.59526981e-02]
 [1.21679870e-01 0.00000000e+00 5.24599575e-08]
 [2.59526981e-02 5.24599575e-08 0.00000000e+00]] 

unweighted covariance matrix of X normalized to nobs-1:  
 [[706.04040404  27.78787879  41.02020202]
 [ 27.78787879  45.06222222  23.81939394]
 [ 41.02020202  23.81939394  48.0589899 ]] 

unweighted standard deviations X normalized to nobs-1:  
 [26.57142081  6.7128401   6.93245915]


We can also use weightedcorrs to find the unweighted correlation coeeficients, covariance, and standard devations, normalized to nobs as follows:

In [7]:
output = weightedcorrs(X,1)

R = output['R']
p = output['p']
wcov = output['wcov']
wstd = output['wstd']

print('unweighted correlation coefficient matrix of X normalized to nobs: ','\n',R,'\n')
print('p-values of the correlation coefficients: ','\n',p,'\n')
print('unweighted covariance matrix of X normalized to nobs: ','\n',wcov,'\n')
print('unweighted standard deviations X normalized to nobs: ','\n',wstd)

unweighted correlation coefficient matrix of X normalized to nobs:  
 [[1.         0.15578811 0.22268743]
 [0.15578811 1.         0.51184337]
 [0.22268743 0.51184337 1.        ]] 

p-values of the correlation coefficients:  
 [[0.00000000e+00 1.21679870e-01 2.59526981e-02]
 [1.21679870e-01 0.00000000e+00 5.24599575e-08]
 [2.59526981e-02 5.24599575e-08 0.00000000e+00]] 

unweighted covariance matrix of X normalized to nobs:  
 [[698.98    27.51    40.61  ]
 [ 27.51    44.6116  23.5812]
 [ 40.61    23.5812  47.5784]] 

unweighted standard deviations X normalized to nobs:  
 [26.4382299   6.67919157  6.89770976]


**Method used to calculate p-values of the correlation coefficients**

The method used to calculate the p-values of the correlation coefficients is the same as the method used by scipy.stats.pearsonr. A beta distribution is used on the interval [-1, 1], with equal shape parameters a = b = nobs/2 - 1. In terms of SciPy’s implementation of the beta distribution, the distribution of r is:

In [8]:
import scipy.stats
nobs = np.shape(X)[0]
dist = scipy.stats.beta(nobs/2 - 1, nobs/2 - 1, loc=-1, scale=2)

The default p-value is a two-sided p-value. For a given sample with correlation coefficient r, the p-value is the probability that abs(r’) of a random sample x’ and y’ drawn from the population with zero correlation would be greater than or equal to abs(r). In terms of the object dist shown above, the p-value for a given r and length n can be computed as:

In [9]:
r = output['R']
p = 2*dist.cdf(-abs(r))
print('p-values of the correlation coefficients: ','\n',p)

p-values of the correlation coefficients:  
 [[0.00000000e+00 1.21679870e-01 2.59526981e-02]
 [1.21679870e-01 0.00000000e+00 5.24599575e-08]
 [2.59526981e-02 5.24599575e-08 0.00000000e+00]]
